In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym
import matplotlib.pyplot as plt
from datetime import datetime
import random
import os

In [2]:
seed = 9295
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [3]:
# Hyper Parameters
HIDDEN_LAYER        = 50
BATCH_SIZE          = 32
LR                  = 1e-4  # learning rate
EPSILON             = 0.9   # greedy policy
GAMMA               = 0.9   # reward discount
TARGET_REPLACE_ITER = 500   # target update frequency
MEMORY_CAPACITY     = 100000
TERMINAL_BIAS       = 0.5   # no. of terminal memories in batch

In [4]:
env = gym.make('CartPole-v0')
env.seed(seed);

In [5]:
XTRA_FEAT   = 0 #masscart, masspole, length
N_ACTIONS   = env.action_space.n 
N_STATES    = env.observation_space.shape[0] + XTRA_FEAT
ENV_A_SHAPE = 0 if isinstance(env.action_space.sample(), int) else env.action_space.sample().shape     # to confirm the shape

In [6]:
class SumTree(object):
    """
    This SumTree code is modified version of Morvan Zhou: 
    https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow/blob/master/contents/5.2_Prioritized_Replay_DQN/RL_brain.py
    """
    data_pointer = 0
    
    """
    Here we initialize the tree with all nodes = 0, and initialize the data with all values = 0
    """
    def __init__(self, capacity):
        self.capacity = capacity # Number of leaf nodes (final nodes) that contains experiences
        
        # Generate the tree with all nodes values = 0
        # To understand this calculation (2 * capacity - 1) look at the schema above
        # Remember we are in a binary node (each node has max 2 children) so 2x size of leaf (capacity) - 1 (root node)
        # Parent nodes = capacity - 1
        # Leaf nodes = capacity
        self.tree = np.zeros(2 * capacity - 1)
        
        """ tree:
            0
           / \
          0   0
         / \ / \
        0  0 0  0  [Size: capacity] it's at this line that there is the priorities score (aka pi)
        """
        
        # Contains the experiences (so the size of data is capacity)
        self.data = np.zeros(capacity, dtype=object)
    
    
    """
    Here we add our priority score in the sumtree leaf and add the experience in data
    """
    def add(self, priority, data):
        # Look at what index we want to put the experience
        tree_index = self.data_pointer + self.capacity - 1
        
        """ tree:
            0
           / \
          0   0
         / \ / \
tree_index  0 0  0  We fill the leaves from left to right
        """
        
        # Update data frame
        self.data[self.data_pointer] = data
        
        # Update the leaf
        self.update (tree_index, priority)
        
        # Add 1 to data_pointer
        self.data_pointer += 1
        
        if self.data_pointer >= self.capacity:  # If we're above the capacity, you go back to first index (we overwrite)
            self.data_pointer = 0
            
    
    """
    Update the leaf priority score and propagate the change through tree
    """
    def update(self, tree_index, priority):
        # Change = new priority score - former priority score
        change = priority - self.tree[tree_index]
        self.tree[tree_index] = priority
        
        # then propagate the change through tree
        while tree_index != 0:    # this method is faster than the recursive loop in the reference code
            
            """
            Here we want to access the line above
            THE NUMBERS IN THIS TREE ARE THE INDEXES NOT THE PRIORITY VALUES
            
                0
               / \
              1   2
             / \ / \
            3  4 5  [6] 
            
            If we are in leaf at index 6, we updated the priority score
            We need then to update index 2 node
            So tree_index = (tree_index - 1) // 2
            tree_index = (6-1)//2
            tree_index = 2 (because // round the result)
            """
            tree_index = (tree_index - 1) // 2
            self.tree[tree_index] += change
    
    
    """
    Here we get the leaf_index, priority value of that leaf and experience associated with that index
    """
    def get_leaf(self, v):
        """
        Tree structure and array storage:
        Tree index:
             0         -> storing priority sum
            / \
          1     2
         / \   / \
        3   4 5   6    -> storing priority for experiences
        Array type for storing:
        [0,1,2,3,4,5,6]
        """
        parent_index = 0
        
        while True: # the while loop is faster than the method in the reference code
            left_child_index = 2 * parent_index + 1
            right_child_index = left_child_index + 1
            
            # If we reach bottom, end the search
            if left_child_index >= len(self.tree):
                leaf_index = parent_index
                break
            
            else: # downward search, always search for a higher priority node
                
                if v <= self.tree[left_child_index]:
                    parent_index = left_child_index
                    
                else:
                    v -= self.tree[left_child_index]
                    parent_index = right_child_index
            
        data_index = leaf_index - self.capacity + 1

        return leaf_index, self.tree[leaf_index], self.data[data_index]
    
    @property
    def total_priority(self):
        return self.tree[0] # Returns the root node

In [7]:
class Memory(object):  # stored as ( s, a, r, s_ ) in SumTree
    """
    This SumTree code is modified version and the original code is from:
    https://github.com/jaara/AI-blog/blob/master/Seaquest-DDQN-PER.py
    """
    PER_e = 0.01  # Hyperparameter that we use to avoid some experiences to have 0 probability of being taken
    PER_a = 0.6  # Hyperparameter that we use to make a tradeoff between taking only exp with high priority and sampling randomly
    PER_b = 0.4  # importance-sampling, from initial value increasing to 1
    
    PER_b_increment_per_sampling = 0.001
    
    absolute_error_upper = 1.  # clipped abs error

    def __init__(self, capacity):
        # Making the tree 
        """
        Remember that our tree is composed of a sum tree that contains the priority scores at his leaf
        And also a data array
        We don't use deque because it means that at each timestep our experiences change index by one.
        We prefer to use a simple array and to overwrite when the memory is full.
        """
        self.tree = SumTree(capacity)
        
    """
    Store a new experience in our tree
    Each new experience have a score of max_prority (it will be then improved when we use this exp to train our DDQN)
    """
    def store(self, experience):
        # Find the max priority
        max_priority = np.max(self.tree.tree[-self.tree.capacity:])
        
        # If the max priority = 0 we can't put priority = 0 since this exp will never have a chance to be selected
        # So we use a minimum priority
        if max_priority == 0:
            max_priority = self.absolute_error_upper
        
        self.tree.add(max_priority, experience)   # set the max p for new p

        
    """
    - First, to sample a minibatch of k size, the range [0, priority_total] is / into k ranges.
    - Then a value is uniformly sampled from each range
    - We search in the sumtree, the experience where priority score correspond to sample values are retrieved from.
    - Then, we calculate IS weights for each minibatch element
    """
    def sample(self, n):
        # Create a sample array that will contains the minibatch
        memory_b = []
        
        b_idx, b_ISWeights = np.empty((n,), dtype=np.int32), np.empty((n, 1), dtype=np.float32)
        
        # Calculate the priority segment
        # Here, as explained in the paper, we divide the Range[0, ptotal] into n ranges
        priority_segment = self.tree.total_priority / n       # priority segment
    
        # Here we increasing the PER_b each time we sample a new minibatch
        self.PER_b = np.min([1., self.PER_b + self.PER_b_increment_per_sampling])  # max = 1
        
        # Calculating the max_weight
        p_min = np.min(self.tree.tree[-self.tree.capacity:]) / self.tree.total_priority
        max_weight = (p_min * n) ** (-self.PER_b)
        
        for i in range(n):
            """
            A value is uniformly sample from each range
            """
            a, b = priority_segment * i, priority_segment * (i + 1)
            value = np.random.uniform(a, b)
            
            """
            Experience that correspond to each value is retrieved
            """
            index, priority, data = self.tree.get_leaf(value)
            
            #P(j)
            sampling_probabilities = priority / self.tree.total_priority
            
            #  IS = (1/N * 1/P(i))**b /max wi == (N*P(i))**-b  /max wi
            b_ISWeights[i, 0] = np.power(n * sampling_probabilities, -self.PER_b)/ max_weight
                                   
            b_idx[i]= index
            
            experience = [data]
            
            memory_b.append(experience)
        
        return b_idx, memory_b, b_ISWeights
    
    """
    Update the priorities on the tree
    """
    def batch_update(self, tree_idx, abs_errors):
        abs_errors += self.PER_e  # convert to abs and avoid 0
        clipped_errors = np.minimum(abs_errors, self.absolute_error_upper)
        ps = np.power(clipped_errors, self.PER_a)

        for ti, p in zip(tree_idx, ps):
            self.tree.update(ti, p)

In [8]:
# # SumTree
# # a binary tree data structure where the parent’s value is the sum of its children
# class SumTree:
#     write = 0

#     def __init__(self, capacity):
#         self.capacity = capacity
#         self.tree = np.zeros(2 * capacity - 1)
#         self.data = np.zeros(capacity, dtype=object)
#         self.n_entries = 0

#     # update to the root node
#     def _propagate(self, idx, change):
#         parent = (idx - 1) // 2

#         self.tree[parent] += change

#         if parent != 0:
#             self._propagate(parent, change)

#     # find sample on leaf node
#     def _retrieve(self, idx, s):
#         left = 2 * idx + 1
#         right = left + 1

#         if left >= len(self.tree):
#             return idx

#         if s <= self.tree[left]:
#             return self._retrieve(left, s)
#         else:
#             return self._retrieve(right, s - self.tree[left])

#     def total(self):
#         return self.tree[0]

#     # store priority and sample
#     def add(self, p, data):
#         idx = self.write + self.capacity - 1

#         self.data[self.write] = data
#         self.update(idx, p)

#         self.write += 1
#         if self.write >= self.capacity:
#             self.write = 0

#         if self.n_entries < self.capacity:
#             self.n_entries += 1

#     # update priority
#     def update(self, idx, p):
#         change = p - self.tree[idx]

#         self.tree[idx] = p
#         self._propagate(idx, change)

#     # get priority and sample
#     def get(self, s):
#         idx = self._retrieve(0, s)
#         dataIdx = idx - self.capacity + 1

#         return (idx, self.tree[idx], self.data[dataIdx])


# class Memory:  # stored as ( s, a, r, s_ ) in SumTree
#     e = 0.01
#     a = 0.6
#     beta = 0.4
#     absolute_error_upper = 1.
#     beta_increment_per_sampling = 0.001

#     def __init__(self, capacity):
#         self.tree = SumTree(capacity)
#         self.capacity = capacity

#     def _get_priority(self, error):
#         return (error + self.e) ** self.a
        

#     def add(self, sample):
#         max_priority = np.max(self.tree.tree[-self.tree.capacity:])
#         if max_priority == 0:
#             max_priority = self.absolute_error_upper
#         self.tree.add(max_priority, sample)

#     def sample(self, n):
#         batch = []
#         idxs = []
#         segment = self.tree.total() / n
#         priorities = []

#         self.beta = np.min([1., self.beta + self.beta_increment_per_sampling])

#         for i in range(n):
#             a = segment * i
#             b = segment * (i + 1)

#             s = random.uniform(a, b)
#             (idx, p, data) = self.tree.get(s)
#             priorities.append(p)
#             batch.append(data)
#             idxs.append(idx)

#         sampling_probabilities = priorities / self.tree.total()
#         is_weight = np.power(self.tree.n_entries * sampling_probabilities, -self.beta)
#         is_weight /= is_weight.max()

#         return batch, idxs, is_weight

#     def update(self, idx, error):
#         p = self._get_priority(error)
#         self.tree.update(idx, p)


In [9]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(N_STATES, HIDDEN_LAYER)
        nn.init.kaiming_uniform_(self.fc1.weight)

        self.adv = nn.Linear(HIDDEN_LAYER, N_ACTIONS)
        nn.init.xavier_uniform_(self.adv.weight) 
    
        self.val = nn.Linear(HIDDEN_LAYER, 1)
        nn.init.xavier_uniform_(self.val.weight)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)

        adv = self.adv(x)
        val = self.val(x)
        
        return val + adv - adv.mean()

In [11]:
class D3QN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()
#         print("Neural net")
#         print(self.eval_net)

        self.learn_step_counter  = 0 # for target updating
#         self.memory_counter      = 0
#         self.memory              = np.zeros((int(MEMORY_CAPACITY), N_STATES * 2 + 2)) # initialize memory
        self.pmemory = Memory(MEMORY_CAPACITY)
#         self.good_memory_counter = 0 # for storing non-terminal memories
#         self.good_memory         = np.zeros((int(MEMORY_CAPACITY/2), N_STATES * 2 + 2)) # initialize memory
        
#         self.bad_memory_counter  = 0 # for storing terminal memories
#         self.bad_memory          = np.zeros((int(MEMORY_CAPACITY/2), N_STATES * 2 + 2)) # initialize memory
        
        self.optimizer           = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func           = nn.MSELoss()
        
        self.running_loss        = 0
        self.loss_rec            = 0 
    def choose_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        # input only one sample
        if np.random.uniform() < EPSILON:   # greedy
            actions_value = self.eval_net.forward(x)
            action = torch.max(actions_value, 1)[1].data.numpy()
            action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index
        else:   # random
            action = np.random.randint(0, N_ACTIONS)
            action = action if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
        return action
    
    def choose_greedy_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        # input only one sample
        actions_value = self.eval_net.forward(x)
        action = torch.max(actions_value, 1)[1].data.numpy()
        action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index
        return action

    def store_transition(self, s, a, r, s_):
        self.pmemory.store([s,a,r,s_])
#         transition = np.hstack((s, [a, r], s_))
#         self.pmemory.add(transition)

    def learn(self):
        # target parameter update
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1
        
        tree_idx, batch, ISWeights_mb = self.pmemory.sample(BATCH_SIZE)
        states_mb = np.array([each[0][0] for each in batch], ndmin=N_STATES)
        actions_mb = np.array([each[0][1] for each in batch])
        rewards_mb = np.array([each[0][2] for each in batch]) 
        next_states_mb = np.array([each[0][3] for each in batch], ndmin=N_STATES)
        
        b_s  = torch.FloatTensor(states_mb)
        b_a  = torch.LongTensor( actions_mb.astype(int))
        b_r  = torch.FloatTensor(rewards_mb)
        b_s_ = torch.FloatTensor(next_states_mb)
        
        
#         mini_batch, idxs, is_weights = self.pmemory.sample(BATCH_SIZE)
#         b_memory = np.array(mini_batch)
#         print(b_memory.shape)
#         b_s  = torch.FloatTensor(b_memory[:, :N_STATES])
#         b_a  = torch.LongTensor( b_memory[:, N_STATES:N_STATES+1].astype(int))
#         b_r  = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])
#         b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])

        # q_eval w.r.t the action in experience
        q_eval   = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        a_eval   = self.eval_net(b_s).max(1)[1].view(BATCH_SIZE, 1) #best action according to eval_net
        q_next   = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        q_target = b_r + GAMMA * q_next.gather(1, a_eval)   # shape (batch, 1)

        errors = torch.abs(q_eval - q_target).data.numpy()
        
        memory.batch_update(tree_idx, errors)

        # update priority
#         for i in range(BATCH_SIZE):
#             idx = idxs[i]
#             self.pmemory.update(idx, errors[i])

        loss = self.loss_func(q_eval, q_target)*torch.from_numpy(is_weights).float()
#         temp = loss.detach()
        
#         self.loss_rec = np.append(self.loss_rec, temp.mean())
        self.optimizer.zero_grad()
        loss.mean().backward()
#         loss.backward()
        self.optimizer.step()

In [19]:
dqn.pmemory.sample(BATCH_SIZE)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: RuntimeWarning: divide by zero encountered in double_scalars


(array([100235, 100712, 101711, 102361, 102509, 103470, 104303, 104609,
        105483, 105652, 106549, 107055, 107490, 108161, 108797, 109360,
        110250, 110972, 111500, 112411, 112931, 113181, 113763, 114691,
        114933, 115703, 116251, 116975, 117881, 118119, 119134, 119832],
       dtype=int32),
 [[[array([-0.02766037,  0.00889004,  0.04657389,  0.04099805]),
    1,
    1.0,
    array([-0.02748257,  0.20331426,  0.04739385, -0.23663426])]],
  [[array([-0.00642754,  0.62715971,  0.00246602, -0.89284784]),
    1,
    1.0,
    array([ 0.00611565,  0.82224812, -0.01539094, -1.18475457])]],
  [[array([ 0.06102853,  0.93739929, -0.01911503, -1.44849281]),
    1,
    1.0,
    array([ 0.07977651,  1.13275096, -0.04808488, -1.74708627])]],
  [[array([ 0.05462264,  1.14638433, -0.10656801, -1.81714006]),
    1,
    1.0,
    array([ 0.07755033,  1.34251806, -0.14291081, -2.14094337])]],
  [[array([ 0.00431512,  0.41714686, -0.06181559, -0.65422442]),
    1,
    1.0,
    array([ 0.012

In [15]:
tree_idx, batch, ISWeights_mb = self.pmemory.sample(BATCH_SIZE)

NameError: name 'tree' is not defined

In [12]:
dqn = D3QN()
NO_OF_EPISODES = 10000

In [13]:
print('\nCollecting experience...')
TIMESTEP_LIMIT = 200
total_time_steps = 0
time_rec = []
learn_start_flag = True
learn_start = 0
solve_metric = 195
upgrade_flag = False
upgrade_counter = 0
state_max  = np.ones(N_STATES)*(-100)
state_min  = np.ones(N_STATES)*(100)
for i_episode in range(NO_OF_EPISODES):
    s = env.reset()
    ep_r = 0
    time_steps = 0
    while True:
#         env.render()
        state_max = np.maximum(state_max,s) 
        state_min = np.minimum(state_min,s)
        time_steps += 1
        total_time_steps += 1
        a = dqn.choose_action(s)

        # take action
        s_, r, done, info = env.step(a)
        if done:
            if time_steps >= TIMESTEP_LIMIT:
                r = 10
            else:
                r = -10
           
        dqn.store_transition(s, a, r, s_)
        
        if i_episode >= 2000:
            dqn.learn()
            if learn_start_flag:
                print("Learning starts from EPISODE: ",i_episode)
                learn_start = i_episode
                learn_start_flag = False
        
        if done:
            time_rec = np.append(time_rec, time_steps)
            break
        s = s_
    
    #if minimum of episode length of last 100 episodes is greater than upgrade_metric=195
    if time_rec[-100:].min() > solve_metric:
        break
#         upgrade_counter += 1
#     else:
#         upgrade_counter = 0
        
#     if upgrade_counter > 110:
#         upgrade_counter = 0
#         print("Upgrading @ EPISODE: ", i_episode)
# #         upgrade_metric *= 2
# #         LR *= 0.1
#         EPSILON += 0.02
        

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: RuntimeWarning: divide by zero encountered in double_scalars


RuntimeError: invalid argument 4: Index tensor must have same dimensions as input tensor at /pytorch/aten/src/TH/generic/THTensorMath.c:581

In [ ]:
WINDOW = 10
lossavg = running_mean(dqn.loss_rec, WINDOW)
plt.plot(dqn.loss_rec,alpha=0.5,color='g')
plt.plot(lossavg)

In [ ]:
WINDOW = 100
ravg = running_mean(time_rec, WINDOW)


fig = plt.figure(figsize = (15,6))

ax2 = fig.add_subplot(2, 1, 1)
data = time_rec
ax2.plot(data, color = 'g',alpha=0.5)
ax2.plot(np.ones_like(data)*1500, 'g--')
ax2.plot(np.ones_like(data)*200, 'r--')

ax2.set_xlabel('Iterations',color = 'k')
ax2.set_ylabel('Time Steps',color = 'g')
ax2.set_ylim([1,2.5e2])
fig.tight_layout()
ax2.grid()


plt.plot(ravg)
plt.show()

In [ ]:
MODELNAME = './models/BASE_F' + datetime.now().strftime("_%H_%M_%S")
print(MODELNAME)

In [ ]:
torch.save(dqn.eval_net.state_dict(), MODELNAME)

In [ ]:
env.close()

In [ ]:
print("TOTAL TIMESTEPS: ", total_time_steps)

In [ ]:
ttest_time_rec = []
for i_episode in range(1000):
    env.length   = 0.5 + np.random.uniform(-0.3,0.3)
#     xtra = [env.length]
    s = env.reset()
#     s = np.append(s, xtra)
    time_steps = 0
    while True:
#         env.render()
        time_steps += 1
        a = dqn.choose_action(s)

        # take action
        s_, r, done, info = env.step(a)
#         s_ = np.append(s_, xtra)

        if done or time_steps >= TIMESTEP_LIMIT:
            ttest_time_rec = np.append(ttest_time_rec, time_steps)
            break
        s = s_

In [ ]:
fig = plt.figure(figsize = (15,6))

ax2 = fig.add_subplot(2, 1, 1)
data = ttest_time_rec
ax2.plot(data, color = 'g')
ax2.plot(np.ones_like(data)*1500, 'g--')
ax2.plot(np.ones_like(data)*200, 'r--')

ax2.set_xlabel('Iterations',color = 'k')
ax2.set_ylabel('Time Steps',color = 'g')
ax2.set_ylim([1,2.5e2])
fig.tight_layout()
ax2.grid()
plt.show()